# Pytket Extensions



Pytket Extensionは様々な企業が開発している量子回路プログラミング言語、量子デバイス、量子シミュレーターと連携することを可能にする拡張機能です。
Azure QuantumとAmazon Braketといった、クラウド経由で追加の量子デバイスやシミュレータにアクセスすることも可能です。

pytketのBackendは、量子回路を処理するためのQPU（Quantum Processing Unit）または量子シミュレータへの接続を表します。
pytketで利用可能なBackendの種類は、以下の通りです。
- QPUs: ショットベースの結果を返す量子コンピュータです。例：QuantinuumBackend.

- クラウドバックエンド: pytketがクラウドプラットフォームと連携し、追加のQPUやシミュレータにアクセスできるようにするバックエンドです。例：BraketBackend.

- エミュレータ: これらは古典コンピュータで量子回路をシミュレートし、ショットベースの結果を生成します。エミュレータはノイズ・モデルを使用し、実際のQPUをエミュレートするための接続性制約を持つ場合もあります。例：IBMQEmulatorBackend.

- ステートベクターシミュレータ: 回路によって準備された純粋な量子状態を計算し、ベクトル/ndarrayを返します。例：ForestStateBackend, AerStateBackend.

- ユニタリーシミュレータ: ユニタリーシミュレータは、回路によって適用されるユニタリーオペレータを計算します。ユニタリー行列/ndarrayが返されます。例：AerUnitaryBackend.

- 密度行列シミュレータ: これらのシミュレータは、回路によって準備される密度行列を計算します。例：CirqDensityMatrixSampleBackend 。

- その他の特殊なシミュレータ: 特定のタイプの回路をシミュレートするための拡張機能があります。例：SimplexBackendはClifford回路をシミュレートするために設計されています。


ここでは
- 量子回路の変換
- 量子回路を量子デバイスや量子シミュレータで実行

### TKETとは (再掲)
- Quantum Software Development Kit
- C++で実装
- pythonモジュール　`pytket`で利用可能
- 最適化コンパイラ：　ユーザーフレンドリーな回路→量子デバイスで実行可能な回路に変換可能
    - Language-agnostic (多くの量子プログラミングフレームワーク(qiskit, Cirq, etc)をサポート)
    - Retagetable (多くの量子デバイス(IBM Quantum, Quantinuum, AWS Braket, etc)をサポート)
    - Circuit Optimisation (量子計算時に生じるデバイスエラーの影響を最小化。デバイス依存＆デバイス非依存のものが実装)
    
<img src="fig/tket1.png" width="750">

#### 参照
- [pytket ドキュメント](https://cqcl.github.io/tket/pytket/api/index.html)
- [pytket ユーザーマニュアル](https://cqcl.github.io/pytket/manual/index.html)
- [t|ket⟩ : A Retargetable Compiler for NISQ Devices](https://arxiv.org/abs/2003.10611)

#### 今回必要となる python パッケージ
Python 3.9.7で動作確認をしています。

|  パッケージ （version） |  概要  |
| :---- | :---- |
|  pytket  |  tketを利用するためのpython モジュール  ( available for python3.7 or higher )|
|  pytket-qiskit  |  qiskitを利用するためのpytket-extension  |
|  pytket-braket  |  Amazon Braketを利用するためのpytket-extension  |

<img src="./fig/tket2.png" width="750">

参考：以下のextensionがpytketに用意されている

|  パッケージ （version） |  概要  |
| :---- | :---- |
|  pytket-qsharp  |  Azure Quantumを利用するためのpytket-extension  |
|  pytket-quantinuum  |  Quantinuumを利用するためのpytket-extension  |

環境にインストールされていない場合は、以下のセルの＃を取り除き、インストールしてください。

In [ ]:
#!pip install pytket
#!pip install pytket-qiskit
#!pip install pytket-braket
#!pip install pytket-qsharp #こちらのパッケージは.net等のインストールが必要です。詳しくは こちら を参照ください。
#!pip install pytket-quantinuum
#!pip install pylatexenc #可視化のためのパッケージ

## 1. 量子回路の変換
pytketでは
- qiskitで記述した量子回路(`qiskit.QuantumCircuit`)からtketの量子回路のクラスに変換が可能
- tketで記述した量子回路からqiskitの量子回路(`qiskit.QuantumCircuit`)のクラスに変換が可能
- tketで記述した量子回路と他の量子プログラミング言語での量子回路の交換が可能

参照：[pytket-extensions](https://cqcl.github.io/pytket-extensions/api/index.html) 

### 1-1. `qiskit`の量子回路から`tket`の量子回路に変換

In [ ]:
from qiskit import QuantumCircuit

qs_bell = QuantumCircuit(2)
qs_bell.h(0)
qs_bell.cx(0,1)
qs_bell.measure_all()

qs_bell.draw(output='mpl')

In [ ]:
from pytket.extensions.qiskit import qiskit_to_tk

In [ ]:
bell2 = qiskit_to_tk(qs_bell)
bell2

In [ ]:
render_circuit_jupyter(bell2)

### 1-2. `tket`の量子回路から`qiskit`の量子回路に変換

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

bell = Circuit(2)
bell.H(0).CX(0,1)
bell.measure_all()

render_circuit_jupyter(bell)

In [ ]:
from pytket.extensions.qiskit import tk_to_qiskit

In [ ]:
qs_bell2 = tk_to_qiskit(bell)
qs_bell2

In [ ]:
qs_bell2.draw('mpl')

### 1-3. `braket`の量子回路から`tket`の量子回路に変換

In [ ]:
from pytket.extensions.braket import tk_to_braket
aws_bell2 = tk_to_braket(bell)
print(aws_bell2[0])

### 1-4. `tket`の量子回路から`braket`の量子回路に変換

In [ ]:
from braket.devices import LocalSimulator
from braket.circuits import Circuit

# create the circuit
aws_bell = Circuit().h(0).cnot(0, 1)
print(aws_bell)

In [ ]:
from pytket.extensions.braket import braket_to_tk
bell3 = braket_to_tk(aws_bell)
render_circuit_jupyter(bell3)

## 2. 量子回路を量子デバイスや量子シミュレータで実行

### 下記のスクリプトはAmazon Braketで<span style="color: red; ">有料の操作</span>となることがありますので、ご注意ください。

### 2-1. pytket-qiskitを利用して、IBM Quantumのシミュレータや実機にジョブを実行

#### a. IBM Quantum へのアクセス設定

In [ ]:
path = 'key/ibm-token'
f = open(path)
ibm_token = f.read()
f.close()
from pytket.extensions.qiskit.backends.config import set_ibmq_config
set_ibmq_config(ibmq_api_token=ibm_token)

In [ ]:
#from qiskit import IBMQ
#IBMQ.save_account(ibm_token, overwrite=True)
#from pytket.extensions.qiskit.backends.config import set_ibmq_config
#set_ibmq_config(ibmq_api_token=ibm_token,hub="ibm-q",group="open",project="main")

In [ ]:
from pytket.extensions.qiskit import IBMQBackend, IBMQEmulatorBackend, AerBackend
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

In [ ]:
list_backend=IBMQBackend.available_devices()
for i in range(len(list_backend)):
    print(list_backend[i].device_name)

#### b. backendの選択

In [ ]:
# ibm_backend = AerBackend() #a.を省略すること可能
# ibm_backend = IBMQBackend("ibmq_lima")
ibm_backend = IBMQBackend("ibmq_qasm_simulator")
# AerBackend to emulate the behaviour of IBMQBackend. Requires a valid IBMQ account.
#ibm_backend = IBMQEmulatorBackend("ibmq_lima")

#### c. backendのゲートセットに書き換える
`Backend.get_compiled_circuit(Circuit)`

In [ ]:
bell = Circuit(2)
bell.H(0).CX(0,1)
bell.measure_all()
render_circuit_jupyter(bell)

ibm_bell = ibm_backend.get_compiled_circuit(bell)
render_circuit_jupyter(ibm_bell)

#### d. 選択したbackendに量子回路を送信して実行させる

In [ ]:
ibm_handle = ibm_backend.process_circuit(ibm_bell, n_shots =1000)

In [ ]:
ibm_result = ibm_backend.get_result(ibm_handle)

In [ ]:
#ibm_backend.cancel(ibm_handle)

#### e. 結果の表示

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(ibm_result.get_counts())

In [ ]:
#print(str(ibm_handle))

In [ ]:
#from pytket.backends import ResultHandle
#from pytket.extensions.qiskit import IBMQBackend
#ibm_backend = IBMQBackend("ibmq_quito")
#ibm_backend = IBMQBackend("ibmq_qasm_simulator")

In [ ]:
#result = ibm_backend.get_result(handle)
#counts = result.get_counts()
#print(counts)
#from qiskit.visualization import plot_histogram
#plot_histogram(counts)

### 2-2. pytket-braketを利用して、Amazon Braketのシミュレータや実機にジョブを実行

#### a-1. ローカル環境からBraketにアクセスされている方は、keyフォルダーのaws-keyのアクセスキー等の情報を入力し下記を実行してください

In [ ]:
#path = 'key/aws-key'
#f = open(path)
#aws_access_key_id, aws_secret_access_key, s3_name, bucket_key= [s.strip() for s in f.readlines()]
#f.close()

In [ ]:
#import boto3
#from braket.aws import AwsSession
#boto_session = boto3.Session(
#    aws_access_key_id= aws_access_key_id,
#    aws_secret_access_key= aws_secret_access_key,
#    region_name= 'us-east-1'
##    region_name= 'us-west-1'
##    region_name= 'eu-west-2'
#)
## us-east-1: IonQ, Simulators (haven't support Xanadu yet)
## us-west-1: Rigetti, Simulators
## eu-west-2: Lucy, Simulators
##
#aws_session = AwsSession(boto_session=boto_session)

#### a-2. Sagemaker StudioからBraketにアクセスされている方は下記を実行してください。

In [ ]:
import boto3
from braket.aws import AwsSession
boto_session = boto3.Session(
    region_name= 'us-east-1'
#    region_name= 'us-west-1'
#    region_name= 'eu-west-2'
)
# us-east-1: IonQ, Simulators (haven't support Xanadu yet)
# us-west-1: Rigetti, Simulators
# eu-west-2: Lucy, Simulators
#
aws_session = AwsSession(boto_session=boto_session)
s3_name = aws_session.default_bucket()
bucket_key = 'pytket/qsrh'

In [ ]:
aws_session.search_devices()

#### b. backend の選択

In [ ]:
from pytket.extensions.braket import BraketBackend

# ionQ   arn:aws:braket:::device/qpu/ionq/ionQdevice, region: us-east-1
# aws_backend = BraketBackend(device='ionQdevice', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'ionq', aws_session=aws_session)
# Aspen-M-1   arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1 #Aspen-M-1, region: us-west-1
# aws_backend = BraketBackend(device='Aspen-M-1',region = 'us-west-1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'rigetti', aws_session=aws_session)
# Aspen-11   arn:aws:braket:::device/qpu/rigetti/Aspen-11 #Aspen-11, region: us-west-1
# aws_backend = BraketBackend(device='Aspen-11', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'rigetti', aws_session=aws_session)
# Oxford Lucy   arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy #Lucy, region: eu-west-2
# aws_backend = BraketBackend(device='Lucy',region = 'eu-west-2', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'oqc', aws_session=aws_session)
# SV1    sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'     #SV1, region:
# aws_backend = BraketBackend(device='sv1', s3_bucket=s3_name , s3_folder = bucket_key, aws_session=aws_session)
# TN1   sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/tn1'    #TN1, region: us-west-2, us-east-1
# aws_backend = BraketBackend(device='tn1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'quantum-simulator', provider = 'amazon', aws_session=aws_session)
# DM1   sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/dm1'    #DM1, region: 
# aws_backend = BraketBackend(device='dm1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'quantum-simulator', provider = 'amazon', aws_session=aws_session)
# Local sv1 simulator
aws_backend = BraketBackend(local = True) #a.を省略すること可能

In [ ]:
#aws_backend.backend_info

#### c. backendのゲートセットに書き換える
`Backend.get_compiled_circuit(Circuit)`

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
# bell状態の作成
bell = Circuit(2)
bell.H(0).CX(0,1)
bell.measure_all()
render_circuit_jupyter(bell)

# backend に用意されているゲートで量子回路を表現
aws_bell = aws_backend.get_compiled_circuit(bell)
render_circuit_jupyter(aws_bell)

#### d. 選択したbackendに量子回路を送信して実行させる

In [ ]:
aws_handle = aws_backend.process_circuit(aws_bell, n_shots =100)

In [ ]:
aws_result = aws_backend.get_result(aws_handle)

#### e. 結果の表示

In [ ]:
aws_counts = aws_result.get_counts()
print(aws_counts)
from qiskit.visualization import plot_histogram
plot_histogram(aws_result.get_counts())

In [ ]:
#print(str(aws_handle))

In [ ]:
#from pytket.backends import ResultHandle
#handle = ResultHandle.from_str("handle_id")

In [ ]:
#result = aws_backend.get_result(handle)
#print(result.get_counts())
#from qiskit.visualization import plot_histogram
#plot_histogram(aws_result.get_counts())